In [191]:
def isStopCodon(codon, stop_codones):
    isStopCodon = False
    for stop_codon in stop_codones:
        if codon.find(stop_codon) != -1:
            isStopCodon = True
    return isStopCodon

def isStartCodon(codon):
    if codon.find("ATG") != -1:
        return True
    return False

In [192]:
from Bio import SeqIO

folder_name = "viruses/data/"
bacterial1_file = "bacterial1.fasta"
bacterial2_file = "bacterial2.fasta"
bacterial3_file = "bacterial3.fasta"
bacterial4_file = "bacterial4.fasta"
mamalian1_file = "mamalian1.fasta"
mamalian2_file = "mamalian2.fasta"
mamalian3_file = "mamalian3.fasta"
mamalian4_file = "mamalian4.fasta"

start_codon = ['ATG']
stop_codones = ['TAG', 'TAA', 'TGA']

record = SeqIO.read(folder_name+bacterial1_file, "fasta")
sequence = record.seq
table = 11
min_pro_len = 2
coding_sequences = []

for strand, sequence in [("forward", record.seq), ("reverse", record.seq.reverse_complement())]:
    for frame in range(3):
        length = (len(record)-frame)
        codones = [sequence[frame+i:frame+i+3] for i in range (0, length, 3)]
        frame_coding_sequences = []
        orf = ""
        write = False
        for codon in codones:
            if isStartCodon(codon):
                write = True
            if isStopCodon(codon, stop_codones) and write:
                orf = orf + codon
                if(len(orf) > 100):
                    frame_coding_sequences.append(orf)
                orf = ""
                write = False
            elif write:
                orf = orf + codon
        coding_sequences.append(frame_coding_sequences)
print(coding_sequences[4])

[Seq('ATGGAAGATAAAGAATTTTTGATTAAAAAAGTAGAATTATTAGAATCAGCAATC...TAA'), Seq('ATGATAACATCTTTTGAATCACTAGCTGAAAGGCGTTTAATAACTCTTAATTAT...TAA'), Seq('ATGGCAATCATCACAGTTACAGCACAAGCGAACGAAAAAAATACACGAACAGTA...TAG'), Seq('ATGTATACAGCAGAAGAGAAAGAACAAATTATCGACATCGTGGATAAGATGAGC...TGA'), Seq('ATGAAAAGAAAATATTTTAATGACAAAAGATATTGCCACTGCTTCGATGTACCA...TAA'), Seq('ATGATATACGCTTCAGCAAAGAAATTTTTGATGAATTATTTGAAGAAGTTTGGC...TAA'), Seq('ATGAGTTATTTTATATCTGTATTCCAGAGCCTCATGATAGAAACGGCTATTTGG...TAG'), Seq('ATGTGTAAGAAACGCAAATACACAAAAATGGGCGCTTTATATTCAATAGCAAAT...TAA'), Seq('ATGACAAATGAAGAATTGTATGAAAGAATTAATAGCATGCTAAAAGAGCAAGGT...TGA'), Seq('ATGACTAAATCACGTTTGAGCTTACCGAATACCGTAGCATTCCCTTATCTTACT...TAA'), Seq('ATGGTAAAACATTTTTTTGTAGAAGAAGATGACAATGGCAAAGAAATTAAGCGA...TAA'), Seq('ATGGCGAATATGATAAAGTTGAAACGCCAGAACGACAAGAACCGCCAAAATTTT...TAA'), Seq('ATGATAAGACAACTGAACAACCTGACACCGAAAGGACGCAAGAAGTTAGCGATG...TAA'), Seq('ATGTTATTTATAAGGAGTCGAGTGGAAACCCTTATGTTACTAATCCGATTGGAT...TGA'), Seq('ATGTCAGCAGTTGTTACTTCGCTAGCTG